In [73]:
# numeric sample (witout lasso and ridge)

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [75]:
from sklearn.preprocessing import StandardScaler ,PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import  Lasso #Ridge #  , LinearRegression
from sklearn.metrics import mean_squared_error

In [76]:
file_path = './Tehran.xlsx'
df = pd.read_excel(file_path)
# Remove rows with at least one NaN
df = df.dropna()
scaler = StandardScaler() # normalizer
# lambda_val = 1
lambdas = [1e4, 1e2, 10 , 1, 1e-2, 1e-4, 1e-8] # ridge coefficient

In [77]:
# df
# df.head(7)
# df.describe()
# print(df['Area'].describe())
# print(df[df['Area'] > 1e6])
# len(df['Address'].unique())
# df.columns
# df.dtypes

In [78]:
# preprocessing
# data cleaning, remove outliers based on IQR: remain first to third quantile +-1.5 IQR
Q1_area = df['Area'].quantile(0.25)
Q3_area = df['Area'].quantile(0.75)
IQR_area = Q3_area - Q1_area

Q1_price = df['PriceUSD'].quantile(0.25)
Q3_price = df['PriceUSD'].quantile(0.75)
IQR_price = Q3_price - Q1_price

lower_bound_area = Q1_area - 1.5 * IQR_area
upper_bound_area = Q3_area + 1.5 * IQR_area

lower_bound_price = Q1_price - 1.5 * IQR_price
upper_bound_price = Q3_price + 1.5 * IQR_price

In [79]:
df_cleaned = df[(df['Area'] >= lower_bound_area) & (df['Area'] <= upper_bound_area) &
                (df['PriceUSD'] >= lower_bound_price) & (df['PriceUSD'] <= upper_bound_price)]

# Check the cleaned dataset
# df_cleaned.describe()
# all to numeric
TF_mapper_text = {'False': 0, 'True': 1}
TF_mapper_bool = {True: 1, False: 0}
YN_mapper = {'No': 0, 'Yes': 1}

df_cleaned ['Has_Parking'] = df_cleaned ['Parking'].map(TF_mapper_bool) # or .replace or .astype(int)
df_cleaned ['Has_Warehouse'] = df_cleaned ['Warehouse'].map(TF_mapper_bool)
df_cleaned ['Has_Elevator'] = df_cleaned ['Elevator'].map(TF_mapper_bool)

X_columns = ['Area', 'Room', 'Has_Parking', 'Has_Warehouse', 'Has_Elevator']
X_scale_cols= ['Area', 'Room']
df_cleaned = pd.concat( [df_cleaned['Address'] , df_cleaned[X_columns] , df_cleaned['PriceUSD']], axis=1) # axis=1 for column merge , axis=0 for row merge

# df_cleaned

C:\Users\mr.rahimy\AppData\Local\Temp\ipykernel_11072\4277749069.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned ['Has_Parking'] = df_cleaned ['Parking'].map(TF_mapper_bool) # or .replace or .astype(int)
C:\Users\mr.rahimy\AppData\Local\Temp\ipykernel_11072\4277749069.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned ['Has_Warehouse'] = df_cleaned ['Warehouse'].map(TF_mapper_bool)
C:\Users\mr.rahimy\AppData\Local\Temp\ipykernel_11072\4277749069.py:13: SettingWithCopyWarning: 
A

In [80]:
# Function to compute the Root Mean Squared Error (RMSE)
def compute_rms_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Function to identify binary features
def identify_binary_features(X):
    initial_features = X.columns

    if not isinstance(X, np.ndarray):
        X = np.array(X)
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    binary_feature_indices = []
    for i in range(X.shape[1]):
        unique_values = np.unique(X[:, i])
        if set(unique_values).issubset({0, 1}):
            binary_feature_indices.append(i)
    return initial_features, binary_feature_indices

def polynomial_features(X, degree):
    # Identify binary features
    initial_features, binary_feature_indices = identify_binary_features(X)

    # Create polynomial features up to the specified degree
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X)

    # Get feature names to identify which columns correspond to binary features
    feature_names = poly.get_feature_names_out()

    # Rename columns related to binary features
    new_feature_names = []
    for name in feature_names:
        parts = name.split()  # Split into individual terms (e.g., ['x0^2', 'x1', 'x2'])
        new_parts = []
        for part in parts:
            # Check if the term is a binary feature with a power (e.g., 'x0^2')
            is_binary_power = False
            for idx in binary_feature_indices:
                binary_namechecker = initial_features[idx]
                if part.startswith(f'{binary_namechecker}^'):
                    is_binary_power = True
                    break
            # If it's a binary feature with a power, replace it with the original binary feature
            if is_binary_power:
                new_parts.append(f'{binary_namechecker}')  # Replace 'x0^2' with 'x0'
            else:
                new_parts.append(part)  # Keep the term as is
        # Join the parts back into a single feature name
        new_name = ' '.join(new_parts)
        new_feature_names.append(new_name)
    # Drop duplicate columns
    df = pd.DataFrame(X_poly, columns=new_feature_names)
    df.columns = new_feature_names
    df = df.loc[:, ~df.columns.duplicated()]
    return df.columns, df.values

    
# Function to perform polynomial regression
def polynomial_regression(X, y, degree, lambda_val):    
    feature, X_poly = polynomial_features(X, degree)
    model = Lasso(alpha=lambda_val)
    model.fit(X_poly, y)
    return feature, model


In [81]:
# Group by 'Address' and perform analysis for each region
addresses = df_cleaned['Address'].unique()
# addresses' lenght is about 200 disstinct values so we fit regression for each of them with acceptable sample size
for address in addresses:
    df_address = df_cleaned[df_cleaned['Address'] == address]
    report ='' # csv or txt for coefficients

    # Skip if not enough data points
    if len(df_address) < 2:
        print(f"Skipping address {address} due to insufficient samples.")
        continue
    X = df_address [X_columns]
    y = df_address['PriceUSD']
    
    # Perform train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train[X_scale_cols] = scaler.fit_transform(X_train[X_scale_cols]) # scaled just continues data
    X_test[X_scale_cols] = scaler.transform(X_test[X_scale_cols])# scaled just continues data
    
    # Skip if train or test sets are empty
    if len(X_train) < 50 or len(X_test) < 25:
        # print(f"Skipping address {address} due to train-test split issues.")
        continue
    else:
      print(f"Processing address: {address}")
    
    # Set polynomial degrees to evaluate
    degrees = [2, 3, 5, 8, 10]
    
    for lambda_val in lambdas:
        train_rms_errors = []
        test_rms_errors = []
        for idx, degree in enumerate(degrees): 
            polylen_traintest_checker=0      
            features, model = polynomial_regression(X_train, y_train, degree, lambda_val)
            (list1, X_train_poly) , (list2, X_test_poly) = (polynomial_features(X_train, degree) , polynomial_features(X_test, degree))

            y_train_pred ,  y_test_pred = (model.predict(X_train_poly), model.predict(X_test_poly))
            train_rms_error, test_rms_error = (compute_rms_error(y_train, y_train_pred), compute_rms_error(y_test, y_test_pred))
            
            # for final plotting per address
            train_rms_errors.append(train_rms_error)
            test_rms_errors.append(test_rms_error)

            # ******************* save transformed and its fitted model coefficients ***********************
            # Retrieve coefficients
            coefficients = model.coef_
            # Combine feature names and coefficients into a DataFrame
            results_df = pd.DataFrame({
                'Feature': features,
                'Coefficient': coefficients
            })
            report = f"Transformed Features and Coefficients{address}: degree={degree}, λ={lambda_val}"
            report += results_df.to_string(index=True) # write df indexes
            with open(f'./Tehran_report/{address}_degree {degree}_,λ {lambda_val}.txt', 'w', encoding='utf-8') as file:
                file.write(report)
            # ******************* end of calculated weights (coefficients) saver ***************************

        # Plot Train RMSE and Test RMSE vs Polynomial Degree for each address
        plt.figure(figsize=(8, 5))
        plt.plot(degrees, train_rms_errors, marker='o', label='Train RMSE', color='blue')
        plt.plot(degrees, test_rms_errors, marker='o', label='Test RMSE', color='red')
        plt.title(f"RMSE vs Degree of Polynomial for {address}, λ={lambda_val}")
        plt.xlabel("Polynomial Degree")
        plt.ylabel("RMSE")
        plt.legend()
        plt.grid(True)
        plt.savefig(f'./Tehran_report/{address}_λ{lambda_val}.jpg', format='jpeg', dpi=300) #instead of display we save it
        plt.close() # to free memory
        # plt.show()

Processing address: Shahran


C:\Users\mr.rahimy\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.772e+07, tolerance: 1.161e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\mr.rahimy\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.684e+10, tolerance: 1.161e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\mr.rahimy\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

Processing address: Pardis


C:\Users\mr.rahimy\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.387e+09, tolerance: 1.977e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\mr.rahimy\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.293e+09, tolerance: 1.977e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\mr.rahimy\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

Processing address: West Ferdows Boulevard


...

Processing address: Parand


C:\Users\mr.rahimy\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.070e+09, tolerance: 1.678e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\mr.rahimy\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.233e+09, tolerance: 1.678e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\mr.rahimy\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

Processing address: Punak


C:\Users\mr.rahimy\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.003e+10, tolerance: 5.565e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\mr.rahimy\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.123e+10, tolerance: 5.565e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\mr.rahimy\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

Skipping address Pakdasht KhatunAbad due to insufficient samples.
Skipping address Chahardangeh due to insufficient samples.
Skipping address Baqershahr due to insufficient samples.
Skipping address Bahar due to insufficient samples.
Skipping address Chardivari due to insufficient samples.
Skipping address Sohanak due to insufficient samples.
Skipping address Zibadasht due to insufficient samples.
Skipping address Chardangeh due to insufficient samples.
Skipping address Villa due to insufficient samples.
Skipping address Azari due to insufficient samples.
Skipping address Shahrake Apadana due to insufficient samples.
Skipping address Darband due to insufficient samples.
Skipping address Eram due to insufficient samples.
Skipping address SabaShahr due to insufficient samples.
Skipping address Shahrake Madaen due to insufficient samples.
Skipping address Salehabad due to insufficient samples.
Skipping address Enghelab due to insufficient samples.
Skipping address Islamshahr Elahieh due t

In [82]:
# from sklearn.inspection import PartialDependenceDisplay

# Assuming `model` is your trained model and `X_train_poly` is your transformed training data
# feature_index = 3  # Index of the 'Area' feature in the transformed data
# PartialDependenceDisplay.from_estimator(model, X_train_poly, features=[feature_index])
# plt.show()